In [282]:

import pandas as pd
import numpy as np
import tqdm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import  GradientBoostingClassifier, VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from skopt import BayesSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

## 데이터 로드

In [283]:
running_df = pd.read_csv(r'C:\Users\User\Desktop\6월11일_마지막\6월14일_유니티_running.csv')
walking_df = pd.read_csv(r'C:\Users\User\Desktop\6월11일_마지막\6월14일_유니티_walking.csv')
jumping_df = pd.read_csv(r'C:\Users\User\Desktop\6월11일_마지막\6월14일_유니티_jumping.csv')

running_df['Activity'] = 'running'
walking_df['Activity'] = 'walking'
jumping_df['Activity'] = 'jumping'

df = pd.concat([running_df, walking_df, jumping_df], axis = 0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89731 entries, 0 to 29824
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Time                  89731 non-null  float64
 1   Gravity Acceleration  89731 non-null  object 
 2   Angular Velocity      89731 non-null  object 
 3   Linear Acceleration   89731 non-null  object 
 4   Activity              89731 non-null  object 
dtypes: float64(1), object(4)
memory usage: 4.1+ MB


In [284]:
crouching_df

,Time,Gravity Acceleration,Angular Velocity,Linear Acceleration,Activity
0,0.02,"(9.73, -1.26, 0.07)","(0.0, 0.0, 0.0)","(0.98, 0.0, 0.01)",crouching
1,0.04,"(9.73, -1.26, 0.07)","(0.0, 0.0, 0.0)","(0.98, 0.0, 0.01)",crouching
2,0.06,"(4.28, -8.79, -0.79)","(-0.23, -4.4, 30.78)","(-2.17, -1.06, -0.07)",crouching
3,0.08,"(4.28, -8.79, -0.79)","(0.0, 0.0, 0.0)","(3.86, -0.37, 0.01)",crouching
4,0.10,"(4.28, -8.79, -0.79)","(0.0, 0.0, 0.0)","(0.91, -0.36, -0.01)",crouching
...,...,...,...,...,...
29820,127.93,"(7.86, -5.84, -0.66)","(-0.06, -1.18, -0.98)","(-3.2, -0.03, 3.52)",crouching
29821,127.95,"(7.96, -5.69, -0.68)","(-0.03, -0.82, -0.67)","(2.93, -18.44, -36.87)",crouching
29822,127.97,"(7.94, -5.72, -0.69)","(-0.01, 0.05, 0.09)","(14.44, -38.8, -98.45)",crouching
29823,127.99,"(7.93, -5.74, -0.7)","(-0.01, 0.03, 0.1)","(-0.13, 3.34, 0.77)",crouching


In [285]:
new_df = df.copy()  
new_df[['Gravity-X', 'Gravity-Y', 'Gravity-Z']] = new_df['Gravity Acceleration'].str.strip('()').str.split(', ', expand=True)
new_df['Gravity-X'] = new_df['Gravity-X'].astype(float)
new_df['Gravity-Y'] = new_df['Gravity-Y'].astype(float)
new_df['Gravity-Z'] = new_df['Gravity-Z'].astype(float)

new_df[['Angular-X', 'Angular-Y', 'Angular-Z']] = new_df['Angular Velocity'].str.strip('()').str.split(', ', expand=True)
new_df['Angular-X'] = new_df['Angular-X'].astype(float)
new_df['Angular-Y'] = new_df['Angular-Y'].astype(float)
new_df['Angular-Z'] = new_df['Angular-Z'].astype(float)

new_df[['Linear-X', 'Linear-Y', 'Linear-Z']] = new_df['Linear Acceleration'].str.strip('()').str.split(', ', expand=True)
new_df['Linear-X'] = new_df['Linear-X'].astype(float)
new_df['Linear-Y'] = new_df['Linear-Y'].astype(float)
new_df['Linear-Z'] = new_df['Linear-Z'].astype(float)

new_df.head()

,Time,Gravity Acceleration,Angular Velocity,Linear Acceleration,Activity,Gravity-X,Gravity-Y,Gravity-Z,Angular-X,Angular-Y,Angular-Z,Linear-X,Linear-Y,Linear-Z
0,0.02,"(9.73, -1.26, 0.07)","(0.0, 0.0, 0.0)","(0.98, 0.0, 0.01)",running,9.73,-1.26,0.07,0.00,0.00,0.00,0.98,0.00,0.01
1,0.04,"(9.73, -1.26, 0.07)","(0.0, 0.0, 0.0)","(0.98, 0.0, 0.01)",running,9.73,-1.26,0.07,0.00,0.00,0.00,0.98,0.00,0.01
2,0.06,"(8.58, -3.46, -3.27)","(-5.62, 4.46, 6.82)","(-2.24, -0.45, -1.19)",running,8.58,-3.46,-3.27,-5.62,4.46,6.82,-2.24,-0.45,-1.19
3,0.08,"(8.58, -3.46, -3.27)","(0.0, 0.0, 0.0)","(3.83, -0.17, -0.36)",running,8.58,-3.46,-3.27,0.00,0.00,0.00,3.83,-0.17,-0.36
4,0.10,"(8.58, -3.46, -3.27)","(0.0, 0.0, 0.0)","(0.89, -0.15, -0.38)",running,8.58,-3.46,-3.27,0.00,0.00,0.00,0.89,-0.15,-0.38


In [286]:
df = new_df.drop(['Gravity Acceleration', 'Angular Velocity', 'Linear Acceleration', 'Time'], axis = 1)

df.head()

,Activity,Gravity-X,Gravity-Y,Gravity-Z,Angular-X,Angular-Y,Angular-Z,Linear-X,Linear-Y,Linear-Z
0,running,9.73,-1.26,0.07,0.00,0.00,0.00,0.98,0.00,0.01
1,running,9.73,-1.26,0.07,0.00,0.00,0.00,0.98,0.00,0.01
2,running,8.58,-3.46,-3.27,-5.62,4.46,6.82,-2.24,-0.45,-1.19
3,running,8.58,-3.46,-3.27,0.00,0.00,0.00,3.83,-0.17,-0.36
4,running,8.58,-3.46,-3.27,0.00,0.00,0.00,0.89,-0.15,-0.38


In [287]:
df = df.drop_duplicates() # 중복값 제거
df = df.sample(frac = 1).reset_index(drop = True) # shuffle하고 index reset

df

,Activity,Gravity-X,Gravity-Y,Gravity-Z,Angular-X,Angular-Y,Angular-Z,Linear-X,Linear-Y,Linear-Z
0,running,8.88,-4.01,1.16,0.84,1.14,-0.09,11.58,27.04,-71.81
1,crouching,7.54,-6.24,-0.69,0.32,-0.09,-0.03,-0.96,-0.72,5.06
2,running,7.93,-3.04,4.91,0.66,-0.14,-0.51,19.15,3.06,-33.13
3,running,8.42,-4.77,1.58,-0.20,-1.08,1.42,4.64,22.87,2.72
4,running,8.88,-2.39,3.42,-1.18,-0.65,-0.17,-40.71,-26.91,75.52
...,...,...,...,...,...,...,...,...,...,...
45003,walking,9.74,-1.00,0.66,0.54,1.21,-0.29,0.00,-0.03,-0.11
45004,running,8.60,-4.72,0.08,0.53,-0.06,-0.94,11.18,-13.46,-182.13
45005,walking,9.77,-0.90,-0.03,0.35,0.30,-0.17,5.15,-67.39,11.92
45006,walking,9.74,-1.02,0.52,-1.52,-0.31,0.04,0.81,-17.04,-2.61


In [288]:
# == 중력 값 포함 ==
gravity_linear_x = df['Gravity-X'] + df['Linear-X']
gravity_linear_y = df['Gravity-Y'] + df['Linear-Y']
gravity_linear_z = df['Gravity-Z'] + df['Linear-Z']

# == Gravity값들은 쓸모 없으므로 버림 == 
df = df.drop(['Gravity-X', 'Gravity-Y', 'Gravity-Z'], axis = 1)

# == 위에서 더한 값들 대입 == 
df['Linear-X'] = gravity_linear_x
df['Linear-Y'] = gravity_linear_y
df['Linear-Z'] = gravity_linear_z 

df.head()

,Activity,Angular-X,Angular-Y,Angular-Z,Linear-X,Linear-Y,Linear-Z
0,running,0.84,1.14,-0.09,20.46,23.03,-70.65
1,crouching,0.32,-0.09,-0.03,6.58,-6.96,4.37
2,running,0.66,-0.14,-0.51,27.08,0.02,-28.22
3,running,-0.20,-1.08,1.42,13.06,18.10,4.30
4,running,-1.18,-0.65,-0.17,-31.83,-29.30,78.94


In [289]:
# 이상치 제거 -> IQR (파생변수 생성 전, 이상치 제거를 하는게 맞음)
def remove_outliers_iqr(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

columns = ['Angular-X', 'Angular-Y', 'Angular-Z', 'Linear-X', 'Linear-Y', 'Linear-Z']

df = remove_outliers_iqr(df, columns)

df.head().reset_index(drop=True)

,Activity,Angular-X,Angular-Y,Angular-Z,Linear-X,Linear-Y,Linear-Z
0,crouching,0.32,-0.09,-0.03,6.58,-6.96,4.37
1,walking,-0.63,0.45,-0.26,16.72,-12.91,-41.69
2,walking,0.63,-0.27,-0.19,10.13,-4.91,12.66
3,crouching,-0.65,1.13,0.07,10.87,7.54,-20.76
4,running,0.56,-1.10,-0.36,7.21,-0.70,-9.22


In [290]:
# 각 축의 크기
def calculate_magnitude(x, y, z):
    return np.sqrt(x**2 + y**2 + z**2)

# Angular 및 Linear 속성의 크기 계산하여 새로운 컬럼 추가
df['Angular-Magnitude'] = df.apply(lambda row: calculate_magnitude(row['Angular-X'], row['Angular-Y'], row['Angular-Z']), axis=1)
df['Linear-Magnitude'] = df.apply(lambda row: calculate_magnitude(row['Linear-X'], row['Linear-Y'], row['Linear-Z']), axis=1)

df.head()

,Activity,Angular-X,Angular-Y,Angular-Z,Linear-X,Linear-Y,Linear-Z,Angular-Magnitude,Linear-Magnitude
1,crouching,0.32,-0.09,-0.03,6.58,-6.96,4.37,0.333766,10.527816
5,walking,-0.63,0.45,-0.26,16.72,-12.91,-41.69,0.816701,46.736309
6,walking,0.63,-0.27,-0.19,10.13,-4.91,12.66,0.711266,16.941092
7,crouching,-0.65,1.13,0.07,10.87,7.54,-20.76,1.305488,24.616785
9,running,0.56,-1.10,-0.36,7.21,-0.70,-9.22,1.285768,11.725293


In [291]:
# 각 축의 변동성 계산
df['Angular-Var'] = df[['Angular-X', 'Angular-Y', 'Angular-Z']].var(axis=1)
df['Linear-Var'] = df[['Linear-X', 'Linear-Y', 'Linear-Z']].var(axis=1)

df.head()

,Activity,Angular-X,Angular-Y,Angular-Z,Linear-X,Linear-Y,Linear-Z,Angular-Magnitude,Linear-Magnitude,Angular-Var,Linear-Var
1,crouching,0.32,-0.09,-0.03,6.58,-6.96,4.37,0.333766,10.527816,0.049033,52.764100
5,walking,-0.63,0.45,-0.26,16.72,-12.91,-41.69,0.816701,46.736309,0.301233,852.992233
6,walking,0.63,-0.27,-0.19,10.13,-4.91,12.66,0.711266,16.941092,0.248133,90.217900
7,crouching,-0.65,1.13,0.07,10.87,7.54,-20.76,1.305488,24.616785,0.801733,302.072633
9,running,0.56,-1.10,-0.36,7.21,-0.70,-9.22,1.285768,11.725293,0.691600,67.517233


In [292]:
# 각 축의 frequency domain
def calculate_fft(data):
    return np.abs(np.fft.rfft(data))

df['Angular-FFT'] = df.apply(lambda row: calculate_fft([row['Angular-X'], row['Angular-Y'], row['Angular-Z']]), axis=1)
df['Linear-FFT'] = df.apply(lambda row: calculate_fft([row['Linear-X'], row['Linear-Y'], row['Linear-Z']]), axis=1)

df.head()

,Activity,Angular-X,Angular-Y,Angular-Z,Linear-X,Linear-Y,Linear-Z,Angular-Magnitude,Linear-Magnitude,Angular-Var,Linear-Var,Angular-FFT,Linear-FFT
1,crouching,0.32,-0.09,-0.03,6.58,-6.96,4.37,0.333766,10.527816,0.049033,52.764100,"[0.2, 0.3835361782153021]","[3.9899999999999993, 12.581426787133482]"
5,walking,-0.63,0.45,-0.26,16.72,-12.91,-41.69,0.816701,46.736309,0.301233,852.992233,"[0.44, 0.9506313691436865]","[37.88000000000001, 50.58632918091606]"
6,walking,0.63,-0.27,-0.19,10.13,-4.91,12.66,0.711266,16.941092,0.248133,90.217900,"[0.16999999999999998, 0.8627861844049196]","[17.88, 16.451556157397388]"
7,crouching,-0.65,1.13,0.07,10.87,7.54,-20.76,1.305488,24.616785,0.801733,302.072633,"[0.5499999999999999, 1.5508707231745653]","[2.349999999999998, 30.10345329027884]"
9,running,0.56,-1.10,-0.36,7.21,-0.70,-9.22,1.285768,11.725293,0.691600,67.517233,"[0.8999999999999999, 1.4404166064024673]","[2.7099999999999973, 14.232065907660772]"


In [293]:
# 가속도와 각속도의 합
df['Total-Magnitude'] = df['Angular-Magnitude'] + df['Linear-Magnitude']

df.head()

,Activity,Angular-X,Angular-Y,Angular-Z,Linear-X,Linear-Y,Linear-Z,Angular-Magnitude,Linear-Magnitude,Angular-Var,Linear-Var,Angular-FFT,Linear-FFT,Total-Magnitude
1,crouching,0.32,-0.09,-0.03,6.58,-6.96,4.37,0.333766,10.527816,0.049033,52.764100,"[0.2, 0.3835361782153021]","[3.9899999999999993, 12.581426787133482]",10.861582
5,walking,-0.63,0.45,-0.26,16.72,-12.91,-41.69,0.816701,46.736309,0.301233,852.992233,"[0.44, 0.9506313691436865]","[37.88000000000001, 50.58632918091606]",47.553010
6,walking,0.63,-0.27,-0.19,10.13,-4.91,12.66,0.711266,16.941092,0.248133,90.217900,"[0.16999999999999998, 0.8627861844049196]","[17.88, 16.451556157397388]",17.652359
7,crouching,-0.65,1.13,0.07,10.87,7.54,-20.76,1.305488,24.616785,0.801733,302.072633,"[0.5499999999999999, 1.5508707231745653]","[2.349999999999998, 30.10345329027884]",25.922273
9,running,0.56,-1.10,-0.36,7.21,-0.70,-9.22,1.285768,11.725293,0.691600,67.517233,"[0.8999999999999999, 1.4404166064024673]","[2.7099999999999973, 14.232065907660772]",13.011061


In [294]:
# 파생변수 다 만듬 -> 결측치 확인
df.isnull().sum()

Activity             0
Angular-X            0
Angular-Y            0
Angular-Z            0
Linear-X             0
Linear-Y             0
Linear-Z             0
Angular-Magnitude    0
Linear-Magnitude     0
Angular-Var          0
Linear-Var           0
Angular-FFT          0
Linear-FFT           0
Total-Magnitude      0
dtype: int64

In [295]:
# Strength와 Phase로 나눠야함 ([Strength, Phase]로 나눠지기 때문)
df['Angular-FFT-Strength'] = df['Angular-FFT'].apply(lambda x: x[0])
df['Angular-FFT-Phase'] = df['Angular-FFT'].apply(lambda x: x[1])
df['Linear-FFT-Strength'] = df['Linear-FFT'].apply(lambda x: x[0])
df['Linear-FFT-Phase'] = df['Linear-FFT'].apply(lambda x: x[1])

# 기존의 FFT 변수를 제거
df.drop(columns=['Angular-FFT', 'Linear-FFT'], inplace=True)

df.head()

,Activity,Angular-X,Angular-Y,Angular-Z,Linear-X,Linear-Y,Linear-Z,Angular-Magnitude,Linear-Magnitude,Angular-Var,Linear-Var,Total-Magnitude,Angular-FFT-Strength,Angular-FFT-Phase,Linear-FFT-Strength,Linear-FFT-Phase
1,crouching,0.32,-0.09,-0.03,6.58,-6.96,4.37,0.333766,10.527816,0.049033,52.764100,10.861582,0.20,0.383536,3.99,12.581427
5,walking,-0.63,0.45,-0.26,16.72,-12.91,-41.69,0.816701,46.736309,0.301233,852.992233,47.553010,0.44,0.950631,37.88,50.586329
6,walking,0.63,-0.27,-0.19,10.13,-4.91,12.66,0.711266,16.941092,0.248133,90.217900,17.652359,0.17,0.862786,17.88,16.451556
7,crouching,-0.65,1.13,0.07,10.87,7.54,-20.76,1.305488,24.616785,0.801733,302.072633,25.922273,0.55,1.550871,2.35,30.103453
9,running,0.56,-1.10,-0.36,7.21,-0.70,-9.22,1.285768,11.725293,0.691600,67.517233,13.011061,0.90,1.440417,2.71,14.232066


In [296]:
# 인덱스 다시 정렬
df = df.reset_index(drop = True)

# 3-axis 지우기
# df = df.drop(['Angular-X', 'Angular-Y', 'Angular-Z', 'Linear-X', 'Linear-Y', 'Linear-Z'], axis = 1)
df = df.drop(['Angular-X', 'Angular-Y', 'Angular-Z', 'Linear-X', 'Linear-Y', 'Linear-Z', 'Linear-Magnitude', 'Linear-Var' ,'Linear-FFT-Strength', 'Linear-FFT-Phase' ,'Total-Magnitude'], axis = 1)

df.head()

,Activity,Angular-Magnitude,Angular-Var,Angular-FFT-Strength,Angular-FFT-Phase
0,crouching,0.333766,0.049033,0.20,0.383536
1,walking,0.816701,0.301233,0.44,0.950631
2,walking,0.711266,0.248133,0.17,0.862786
3,crouching,1.305488,0.801733,0.55,1.550871
4,running,1.285768,0.691600,0.90,1.440417


In [297]:
# == IMU를 통해 가져온 walking 전처리한 것들 csv파일 불러오기
test1 = pd.read_csv(r'C:\Users\User\Desktop\6월11일_마지막\코드들\6월14일_IMU_walking_cleaned.csv')
test2 = pd.read_csv(r'C:\Users\User\Desktop\6월11일_마지막\코드들\6월14일_IMU_running_cleaned.csv')
test3 = pd.read_csv(r'')

test = pd.concat([test1, test2, test3], axis = 0)

test = test.drop(['Unnamed: 0','Angular-X', 'Angular-Y', 'Angular-Z', 'Linear-X', 'Linear-Y', 'Linear-Z', 'Linear-Magnitude', 'Linear-Var' ,'Linear-FFT-Strength', 'Linear-FFT-Phase', 'Total-Magnitude'], axis = 1)

test.head()

,Activity,Angular-Magnitude,Angular-Var,Angular-FFT-Strength,Angular-FFT-Phase
0,walking,0.254403,0.032206,0.030467,0.310833
1,walking,0.265505,0.035226,0.010976,0.325083
2,walking,0.273865,0.037244,0.039281,0.334263
3,walking,0.255958,0.031266,0.094590,0.306265
4,walking,0.274851,0.027905,0.243313,0.289334


In [298]:
X_train = df.drop(columns=['Activity'])
y_train = df['Activity']


X_test = test.drop(columns = ['Activity'])
y_test = test['Activity']

In [299]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), X_train.columns)
    ]
)

# 모델 정의 (하이퍼파라미터 기본 설정)
lgbm_clf = LGBMClassifier(random_state=42)
catboost_clf = CatBoostClassifier(random_state=42, verbose=0)
gb_clf = GradientBoostingClassifier(random_state=42)

# 앙상블 VotingClassifier 정의
ensemble_clf = VotingClassifier(
    estimators=[
        ('lgbm', lgbm_clf),
        ('catboost', catboost_clf),
        ('gb', gb_clf)
    ],
    voting='soft'
)

# 최종 파이프라인 정의
final_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', ensemble_clf)
])

# 최종 모델 학습
final_pipeline.fit(X_train, y_train)

# 예측
y_pred = final_pipeline.predict(X_test)

# 정확도 평가
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

# Confusion Matrix 출력
conf_matrix = confusion_matrix(y_test, y_pred)
print(f'Confusion Matrix:\n{conf_matrix}')


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000455 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1020
[LightGBM] [Info] Number of data points in the train set: 24855, number of used features: 4
[LightGBM] [Info] Start training from score -0.997013
[LightGBM] [Info] Start training from score -1.776309
[LightGBM] [Info] Start training from score -0.772714
Accuracy: 0.6050
Confusion Matrix:
[[ 430  517  915]
 [  45 1994  134]
 [ 444  288 1165]]


In [300]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

# Confusion Matrix 출력
conf_matrix = confusion_matrix(y_test, y_pred)
print(f'Confusion Matrix:\n{conf_matrix}')

Accuracy: 0.6050
Confusion Matrix:
[[ 430  517  915]
 [  45 1994  134]
 [ 444  288 1165]]
